In [1]:
#Loading essential libraries
import numpy as np
import pandas as pd

In [2]:
#Loading datasets
tr = pd.read_csv('train.csv')
camp = pd.read_csv('campaign_data.csv')
coup = pd.read_csv('coupon_item_mapping.csv')
item = pd.read_csv('item_data.csv')
cust_dem = pd.read_csv('customer_demographics.csv')
cust_trans = pd.read_csv('customer_transaction_data.csv')
ts = pd.read_csv('test_QyjYwdj.csv')

In [3]:
#Loading cutomer level aggreagtion data which we prepared earlier
cust_agg = pd.read_csv('cust_agg.csv')

In [4]:
#Deriving dayofweek, month, quarter, year from date in cust_trans
cust_trans['date'] = pd.to_datetime(cust_trans['date'], infer_datetime_format=True)
cust_trans['date_weekday'] = cust_trans.date.dt.weekday_name
cust_trans['date_month'] = cust_trans.date.dt.month
cust_trans['date_quarter'] = cust_trans.date.dt.quarter
cust_trans['date_year'] = cust_trans.date.dt.year

#Also changing negative foramt in discount to positive
cust_trans['other_discount'] = abs(cust_trans['other_discount'])
cust_trans['coupon_discount'] = abs(cust_trans['coupon_discount'])
cust_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324566 entries, 0 to 1324565
Data columns (total 11 columns):
date               1324566 non-null datetime64[ns]
customer_id        1324566 non-null int64
item_id            1324566 non-null int64
quantity           1324566 non-null int64
selling_price      1324566 non-null float64
other_discount     1324566 non-null float64
coupon_discount    1324566 non-null float64
date_weekday       1324566 non-null object
date_month         1324566 non-null int64
date_quarter       1324566 non-null int64
date_year          1324566 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(6), object(1)
memory usage: 111.2+ MB


In [15]:
#Merging coupon and item data
coup_item = coup.merge(item, on='item_id', how='left')
coup_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92663 entries, 0 to 92662
Data columns (total 5 columns):
coupon_id     92663 non-null int64
item_id       92663 non-null int64
brand         92663 non-null int64
brand_type    92663 non-null object
category      92663 non-null object
dtypes: int64(3), object(2)
memory usage: 4.2+ MB


In [6]:
#Merging cust_trans and item data
cust_item = cust_trans.merge(item, on='item_id', how='left')
cust_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1324566 entries, 0 to 1324565
Data columns (total 14 columns):
date               1324566 non-null datetime64[ns]
customer_id        1324566 non-null int64
item_id            1324566 non-null int64
quantity           1324566 non-null int64
selling_price      1324566 non-null float64
other_discount     1324566 non-null float64
coupon_discount    1324566 non-null float64
date_weekday       1324566 non-null object
date_month         1324566 non-null int64
date_quarter       1324566 non-null int64
date_year          1324566 non-null int64
brand              1324566 non-null int64
brand_type         1324566 non-null object
category           1324566 non-null object
dtypes: datetime64[ns](1), float64(3), int64(7), object(3)
memory usage: 151.6+ MB


In [32]:
coup_agg = coup_item.groupby(['coupon_id']).agg({'item_id':['count'], 'brand':['count', 'nunique']})
coup_agg.columns=['coup_' + '_'.join(col).strip() for col in coup_agg.columns.values]
coup_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1116 entries, 1 to 1116
Data columns (total 3 columns):
coup_item_id_count    1116 non-null int64
coup_brand_count      1116 non-null int64
coup_brand_nunique    1116 non-null int64
dtypes: int64(3)
memory usage: 34.9 KB


#### Aggregating and creating Features from transactional data for each item

In [7]:
#Creating item aggregation dataset
#Number of purchase transactions & quantities purchased for each item?
#Total selling price, coupon discount, other discount for each item?
#Number of unique customers who purchased the item?
#How much coupon discount & other discount in price happened for each item?

agg_col = {'customer_id': ['count', 'nunique'], 'quantity':['sum'], 'selling_price':['sum'], 'coupon_discount':['sum'],
          'other_discount':['sum']}

item_agg = cust_item.groupby(['item_id']).agg(agg_col)
item_agg.columns=['item_' + '_'.join(col).strip() for col in item_agg.columns.values]
item_agg.rename(columns={'item_customer_id_count':'item_trans_count'}, inplace=True)
item_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74063 entries, 1 to 74066
Data columns (total 6 columns):
item_trans_count            74063 non-null int64
item_customer_id_nunique    74063 non-null int64
item_quantity_sum           74063 non-null int64
item_selling_price_sum      74063 non-null float64
item_coupon_discount_sum    74063 non-null float64
item_other_discount_sum     74063 non-null float64
dtypes: float64(3), int64(3)
memory usage: 4.0 MB


In [8]:
#Aggregation of item data by weekday in customer's transaction date
for x in cust_item['date_weekday'].unique():
    #Number of purchase transactions & quantities purchased in each month, dayofweek, quarter & year in each item?
    tmp_map = cust_item[(cust_item['date_weekday']==x)].groupby(['item_id']).size()
    cname1 = 'item_'+str(x)+'_trans_count'
    item_agg[cname1] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname1].isnull(), cname1] = 0
    
    tmp_map = cust_item[(cust_item['date_weekday']==x)].groupby(['item_id'])['quantity'].sum()
    cname2 = 'item_'+str(x)+'_quantity_sum'
    item_agg[cname2] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname2].isnull(), cname2] = 0
    
    #Total selling price, coupon discount, other discount across each month, dayofweek, quarter & year in each item?
    tmp_map = cust_item[(cust_item['date_weekday']==x)].groupby(['item_id'])['selling_price'].sum()
    cname3 = 'item_'+str(x)+'_selling_price_sum'
    item_agg[cname3] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname3].isnull(), cname3] = 0
    
    tmp_map = cust_item[(cust_item['date_weekday']==x)].groupby(['item_id'])['coupon_discount'].sum()
    cname4 = 'item_'+str(x)+'_coupon_discount_sum'
    item_agg[cname4] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname4].isnull(), cname4] = 0
    
    tmp_map = cust_item[(cust_item['date_weekday']==x)].groupby(['item_id'])['other_discount'].sum()
    cname5 = 'item_'+str(x)+'_other_discount_sum'
    item_agg[cname5] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname5].isnull(), cname5] = 0
    
    #Number of coupon discount transactions & quantities purchased across each month, dayofweek, quarter & year in each item?
    tmp_map = cust_item[(cust_item['date_weekday']==x) & (cust_item['coupon_discount']!=0)].groupby(['item_id']).size()
    cname6 = 'item_'+str(x)+'_coupdisc_trans_count'
    item_agg[cname6] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname6].isnull(), cname6] = 0
    
    tmp_map = cust_item[(cust_item['date_weekday']==x) & (cust_item['coupon_discount']!=0)].groupby(['item_id'])['quantity'].sum()
    cname7 = 'item_'+str(x)+'_coupdisc_quantity_sum'
    item_agg[cname7] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname7].isnull(), cname7] = 0
    
    #Number of Other discount transactions & quantities purchased across each month, dayofweek, quarter & year in each item?
    tmp_map = cust_item[(cust_item['date_weekday']==x) & (cust_item['other_discount']!=0)].groupby(['item_id']).size()
    cname8 = 'item_'+str(x)+'_otherdisc_trans_count'
    item_agg[cname8] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname8].isnull(), cname8] = 0
    
    tmp_map = cust_item[(cust_item['date_weekday']==x) & (cust_item['other_discount']!=0)].groupby(['item_id'])['quantity'].sum()
    cname9 = 'item_'+str(x)+'_otherdisc_quantity_sum'
    item_agg[cname9] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname9].isnull(), cname9] = 0
    
    #Number of discount transactions & quantities purchased across each month, dayofweek, quarter & year in each item?
    tmp_map = cust_item[(cust_item['date_weekday']==x) &
                        ((cust_item['other_discount']!=0) | (cust_item['coupon_discount']!=0))].groupby(['item_id']).size()
    cname10 = 'item_'+str(x)+'_discount_trans_count'
    item_agg[cname10] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname10].isnull(), cname10] = 0
    
    tmp_map = cust_item[(cust_item['date_weekday']==x) &
                        ((cust_item['other_discount']!=0) | (cust_item['coupon_discount']!=0))].groupby(['item_id'])['quantity'].sum()
    cname11 = 'item_'+str(x)+'_discount_quantity_sum'
    item_agg[cname11] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname11].isnull(), cname11] = 0

In [9]:
#Aggregation of item data by month in customer's transaction date
for x in cust_item['date_month'].unique():
    
    tmp_map = cust_item[(cust_item['date_month']==x)].groupby(['item_id']).size()
    cname1 = 'item_month_'+str(x)+'_trans_count'
    item_agg[cname1] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname1].isnull(), cname1] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x)].groupby(['item_id'])['quantity'].sum()
    cname2 = 'item_month_'+str(x)+'_quantity_sum'
    item_agg[cname2] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname2].isnull(), cname2] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x)].groupby(['item_id'])['selling_price'].sum()
    cname3 = 'item_month_'+str(x)+'_selling_price_sum'
    item_agg[cname3] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname3].isnull(), cname3] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x)].groupby(['item_id'])['coupon_discount'].sum()
    cname4 = 'item_month_'+str(x)+'_coupon_discount_sum'
    item_agg[cname4] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname4].isnull(), cname4] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x)].groupby(['item_id'])['other_discount'].sum()
    cname5 = 'item_month_'+str(x)+'_other_discount_sum'
    item_agg[cname5] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname5].isnull(), cname5] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x) & (cust_item['coupon_discount']!=0)].groupby(['item_id']).size()
    cname6 = 'item_month_'+str(x)+'_coupdisc_trans_count'
    item_agg[cname6] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname6].isnull(), cname6] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x) & (cust_item['coupon_discount']!=0)].groupby(['item_id'])['quantity'].sum()
    cname7 = 'item_month_'+str(x)+'_coupdisc_quantity_sum'
    item_agg[cname7] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname7].isnull(), cname7] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x) & (cust_item['other_discount']!=0)].groupby(['item_id']).size()
    cname8 = 'item_month_'+str(x)+'_otherdisc_trans_count'
    item_agg[cname8] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname8].isnull(), cname8] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x) & (cust_item['other_discount']!=0)].groupby(['item_id'])['quantity'].sum()
    cname9 = 'item_month_'+str(x)+'_otherdisc_quantity_sum'
    item_agg[cname9] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname9].isnull(), cname9] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x) &
                        ((cust_item['other_discount']!=0) | (cust_item['coupon_discount']!=0))].groupby(['item_id']).size()
    cname10 = 'item_month_'+str(x)+'_discount_trans_count'
    item_agg[cname10] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname10].isnull(), cname10] = 0
    
    tmp_map = cust_item[(cust_item['date_month']==x) &
                        ((cust_item['other_discount']!=0) | (cust_item['coupon_discount']!=0))].groupby(['item_id'])['quantity'].sum()
    cname11 = 'item_month_'+str(x)+'_discount_quantity_sum'
    item_agg[cname11] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname11].isnull(), cname11] = 0

In [10]:
#Aggregation of item data by quarter in customer's transaction date
for x in cust_item['date_quarter'].unique():
    
    tmp_map = cust_item[(cust_item['date_quarter']==x)].groupby(['item_id']).size()
    cname1 = 'item_quarter_'+str(x)+'_trans_count'
    item_agg[cname1] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname1].isnull(), cname1] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x)].groupby(['item_id'])['quantity'].sum()
    cname2 = 'item_quarter_'+str(x)+'_quantity_sum'
    item_agg[cname2] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname2].isnull(), cname2] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x)].groupby(['item_id'])['selling_price'].sum()
    cname3 = 'item_quarter_'+str(x)+'_selling_price_sum'
    item_agg[cname3] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname3].isnull(), cname3] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x)].groupby(['item_id'])['coupon_discount'].sum()
    cname4 = 'item_quarter_'+str(x)+'_coupon_discount_sum'
    item_agg[cname4] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname4].isnull(), cname4] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x)].groupby(['item_id'])['other_discount'].sum()
    cname5 = 'item_quarter_'+str(x)+'_other_discount_sum'
    item_agg[cname5] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname5].isnull(), cname5] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x) & (cust_item['coupon_discount']!=0)].groupby(['item_id']).size()
    cname6 = 'item_quarter_'+str(x)+'_coupdisc_trans_count'
    item_agg[cname6] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname6].isnull(), cname6] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x) & (cust_item['coupon_discount']!=0)].groupby(['item_id'])['quantity'].sum()
    cname7 = 'item_quarter_'+str(x)+'_coupdisc_quantity_sum'
    item_agg[cname7] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname7].isnull(), cname7] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x) & (cust_item['other_discount']!=0)].groupby(['item_id']).size()
    cname8 = 'item_quarter_'+str(x)+'_otherdisc_trans_count'
    item_agg[cname8] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname8].isnull(), cname8] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x) & (cust_item['other_discount']!=0)].groupby(['item_id'])['quantity'].sum()
    cname9 = 'item_quarter_'+str(x)+'_otherdisc_quantity_sum'
    item_agg[cname9] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname9].isnull(), cname9] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x) &
                        ((cust_item['other_discount']!=0) | (cust_item['coupon_discount']!=0))].groupby(['item_id']).size()
    cname10 = 'item_quarter_'+str(x)+'_discount_trans_count'
    item_agg[cname10] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname10].isnull(), cname10] = 0
    
    tmp_map = cust_item[(cust_item['date_quarter']==x) &
                        ((cust_item['other_discount']!=0) | (cust_item['coupon_discount']!=0))].groupby(['item_id'])['quantity'].sum()
    cname11 = 'item_quarter_'+str(x)+'_discount_quantity_sum'
    item_agg[cname11] = item_agg.index.map(tmp_map)
    item_agg.loc[item_agg[cname11].isnull(), cname11] = 0

In [37]:
item_agg.reset_index(inplace=True)
item_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74063 entries, 0 to 74062
Columns: 260 entries, item_id to item_quarter_4_discount_quantity_sum
dtypes: float64(256), int64(4)
memory usage: 146.9 MB


#### Aggregating and creating Features from transactional data for each coupon

In [11]:
agg_col = {}

for k in item_agg.columns:
    if k!='item_id':
        agg_col[k] = ['sum']
coup_item_agg = coup.merge(item_agg, on='item_id', how='left')

coup_agg = coup_item_agg.groupby(['coupon_id']).agg(agg_col)
coup_agg.columns=[col[0].replace('item_', 'coup_',1) for col in coup_agg.columns.values]

#Number of unique campaigns & customers targetted for each coupon?
tmp_map = pd.concat([tr, ts], sort=False, axis='rows').groupby(['coupon_id'])['campaign_id'].nunique()
coup_agg['coup_target_campaign_nunique'] = coup_agg.index.map(tmp_map)
tmp_map = pd.concat([tr, ts], sort=False, axis='rows').groupby(['coupon_id'])['customer_id'].nunique()
coup_agg['coup_target_customer_nunique'] = coup_agg.index.map(tmp_map)

#Number of items in each coupon
tmp_map = coup.groupby(['coupon_id']).size()
coup_agg['coup_item_count'] = coup_agg.index.map(tmp_map)

#Number of unique brands in each coupon?
tmp_map = coup_item.groupby(['coupon_id'])['brand'].nunique()
coup_agg['coup_brand_nunique'] = coup_agg.index.map(tmp_map)

In [12]:
#Aggregation of coupon data by Local and Established brand types
for x in coup_item['brand_type'].unique():
    #Number of Local and Established items in each coupon?
    tmp_map = coup_item[(coup_item['brand_type']==x)].groupby(['coupon_id']).size()
    cname1 = 'coup_'+str(x)+'_item_count'
    coup_agg[cname1] = coup_agg.index.map(tmp_map)
    coup_agg.loc[coup_agg[cname1].isnull(), cname1] = 0
    
    #Number of Local and Established unique brands in each coupon?
    tmp_map = coup_item[(coup_item['brand_type']==x)].groupby(['coupon_id'])['brand'].nunique()
    cname2 = 'coup_'+str(x)+'_brand_nunique'
    coup_agg[cname2] = coup_agg.index.map(tmp_map)
    coup_agg.loc[coup_agg[cname2].isnull(), cname1] = 0

In [13]:
tmp = pd.DataFrame(columns=['total','unique'])
for x in coup.coupon_id.unique():
    #Number of unique customers who purchased the items in each coupon?
    x1 = coup[coup['coupon_id']==x]['item_id'].unique()
    t = cust_item[cust_item['item_id'].isin(x1)]['customer_id'].count()
    u = cust_item[cust_item['item_id'].isin(x1)]['customer_id'].nunique()
    tmp=pd.concat([tmp, pd.DataFrame({'total':t,'unique':u}, index=[x])])

coup_agg['coup_customer_trans_nunique'] = coup_agg.index.map(tmp['unique'])

In [17]:
#Aggregation of coupon data by 19 item categories
for x in coup_item['category'].unique():
    #Number of unique items bought in each 19 item categories by each customer?
    tmp_map = coup_item[(coup_item['category']==x)].groupby(['coupon_id'])['item_id'].nunique()
    cname1 = 'coup_'+str(x)+'_item_nunique'
    coup_agg[cname1] = coup_agg.index.map(tmp_map)
    coup_agg.loc[coup_agg[cname1].isnull(), cname1] = 0
    
    #Number of unique brand bought in each 19 item categories by each customer?
    tmp_map = coup_item[(coup_item['category']==x)].groupby(['coupon_id'])['brand'].nunique()
    cname2 = 'coup_'+str(x)+'_brand_nunique'
    coup_agg[cname2] = coup_agg.index.map(tmp_map)
    coup_agg.loc[coup_agg[cname2].isnull(), cname2] = 0
    
    #Number of items & unique brands in Local and established brand types across 19 categories in each coupon?
    for x1 in coup_item['brand_type'].unique():
        tmp_map = coup_item[(coup_item['category']==x)&(coup_item['brand_type']==x1)].groupby(['coupon_id'])['item_id'].nunique()
        cname3 = 'coup_'+str(x1)+'_'+str(x)+'_item_nunique'
        coup_agg[cname3] = coup_agg.index.map(tmp_map)
        coup_agg.loc[coup_agg[cname3].isnull(), cname3] = 0
        
        tmp_map = coup_item[(coup_item['category']==x)&(coup_item['brand_type']==x1)].groupby(['coupon_id'])['brand'].nunique()
        cname4 = 'coup_'+str(x1)+'_'+str(x)+'_brand_nunique'
        coup_agg[cname4] = coup_agg.index.map(tmp_map)
        coup_agg.loc[coup_agg[cname4].isnull(), cname4] = 0       
        

In [18]:
#Coupon discount percent, other discount percent, total discount percent in price for each coupon?
coup_agg['coup_coupdisc_price_perc'] = coup_agg.apply(lambda x: 0 if (x['coup_selling_price_sum']==0)
                                                     else x['coup_coupon_discount_sum']/x['coup_selling_price_sum'], axis=1)

coup_agg['coup_otherdisc_price_perc'] = coup_agg.apply(lambda x: 0 if (x['coup_selling_price_sum']==0)
                                                     else x['coup_other_discount_sum']/x['coup_selling_price_sum'], axis=1)

coup_agg['coup_discount_price_perc'] = coup_agg.apply(lambda x: 0 if (x['coup_selling_price_sum']==0) else
                                                      (x['coup_other_discount_sum']+x['coup_coupon_discount_sum'])/x['coup_selling_price_sum'], axis=1)

In [19]:
#Aggregation of coupon data by weekday in customer's transaction date
for x in cust_item['date_weekday'].unique():
    x1 = 'coup_'
    cname1 = x1+str(x)+'_trans_count'
    cname2 = x1+str(x)+'_quantity_sum'
    cname3 = x1+str(x)+'_coupdisc_trans_count'
    cname4 = x1+str(x)+'_coupdisc_quantity_sum'
    cname5 = x1+str(x)+'_discount_trans_count'
    cname6 = x1+str(x)+'_discount_quantity_sum'
    cname7 = x1+str(x)+'_otherdisc_trans_count'
    cname8 = x1+str(x)+'_otherdisc_quantity_sum'
    cname9 = x1+str(x)+'_selling_price_sum'
    cname10 = x1+str(x)+'_coupon_discount_sum'
    cname11 = x1+str(x)+'_other_discount_sum'
    
    #Coupon discount percent, other discount percent, total discount percent in price in each month, dayofweek, quarter & year in each coupon?
    
    cname12 = x1+str(x)+'_coupdisc_trans_perc'
    cname13 = x1+str(x)+'_discount_trans_perc'
    cname14 = x1+str(x)+'_otherdisc_trans_perc'
    cname15 = x1+str(x)+'_coupdisc_quantity_perc'
    cname16 = x1+str(x)+'_discount_quantity_perc'
    cname17 = x1+str(x)+'_otherdisc_quantity_perc'
    cname18 = x1+str(x)+'_coupdisc_price_perc'
    cname19 = x1+str(x)+'_otherdisc_price_perc'
    cname20 = x1+str(x)+'_discount_price_perc'
    
    coup_agg[cname12] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname3]/x[cname1], axis=1)
    coup_agg[cname13] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname5]/x[cname1], axis=1)
    coup_agg[cname14] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname7]/x[cname1], axis=1)
    coup_agg[cname15] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname4]/x[cname2], axis=1)
    coup_agg[cname16] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname6]/x[cname2], axis=1)
    coup_agg[cname17] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname8]/x[cname2], axis=1)
    coup_agg[cname18] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else x[cname10]/x[cname9], axis=1)
    coup_agg[cname19] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else x[cname11]/x[cname9], axis=1)
    coup_agg[cname20] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else (x[cname10]+x[cname11])/x[cname9], axis=1)

In [20]:
#Aggregation of coupon data by month in customer's transaction date
for x in cust_item['date_month'].unique():
    x1 = 'coup_month_'
    cname1 = x1+str(x)+'_trans_count'
    cname2 = x1+str(x)+'_quantity_sum'
    cname3 = x1+str(x)+'_coupdisc_trans_count'
    cname4 = x1+str(x)+'_coupdisc_quantity_sum'
    cname5 = x1+str(x)+'_discount_trans_count'
    cname6 = x1+str(x)+'_discount_quantity_sum'
    cname7 = x1+str(x)+'_otherdisc_trans_count'
    cname8 = x1+str(x)+'_otherdisc_quantity_sum'
    cname9 = x1+str(x)+'_selling_price_sum'
    cname10 = x1+str(x)+'_coupon_discount_sum'
    cname11 = x1+str(x)+'_other_discount_sum'
    
    
    
    cname12 = x1+str(x)+'_coupdisc_trans_perc'
    cname13 = x1+str(x)+'_discount_trans_perc'
    cname14 = x1+str(x)+'_otherdisc_trans_perc'
    cname15 = x1+str(x)+'_coupdisc_quantity_perc'
    cname16 = x1+str(x)+'_discount_quantity_perc'
    cname17 = x1+str(x)+'_otherdisc_quantity_perc'
    cname18 = x1+str(x)+'_coupdisc_price_perc'
    cname19 = x1+str(x)+'_otherdisc_price_perc'
    cname20 = x1+str(x)+'_discount_price_perc'
    
    coup_agg[cname12] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname3]/x[cname1], axis=1)
    coup_agg[cname13] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname5]/x[cname1], axis=1)
    coup_agg[cname14] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname7]/x[cname1], axis=1)
    coup_agg[cname15] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname4]/x[cname2], axis=1)
    coup_agg[cname16] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname6]/x[cname2], axis=1)
    coup_agg[cname17] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname8]/x[cname2], axis=1)
    coup_agg[cname18] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else x[cname10]/x[cname9], axis=1)
    coup_agg[cname19] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else x[cname11]/x[cname9], axis=1)
    coup_agg[cname20] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else (x[cname10]+x[cname11])/x[cname9], axis=1)

In [21]:
#Aggregation of coupon data by quarter in customer's transaction date
for x in cust_item['date_quarter'].unique():
    x1 = 'coup_quarter_'
    cname1 = x1+str(x)+'_trans_count'
    cname2 = x1+str(x)+'_quantity_sum'
    cname3 = x1+str(x)+'_coupdisc_trans_count'
    cname4 = x1+str(x)+'_coupdisc_quantity_sum'
    cname5 = x1+str(x)+'_discount_trans_count'
    cname6 = x1+str(x)+'_discount_quantity_sum'
    cname7 = x1+str(x)+'_otherdisc_trans_count'
    cname8 = x1+str(x)+'_otherdisc_quantity_sum'
    cname9 = x1+str(x)+'_selling_price_sum'
    cname10 = x1+str(x)+'_coupon_discount_sum'
    cname11 = x1+str(x)+'_other_discount_sum'
    
    #Coupon discount percent, other discount percent, total discount percent in price in each month, dayofweek, quarter & year in each coupon?
    
    cname12 = x1+str(x)+'_coupdisc_trans_perc'
    cname13 = x1+str(x)+'_discount_trans_perc'
    cname14 = x1+str(x)+'_otherdisc_trans_perc'
    cname15 = x1+str(x)+'_coupdisc_quantity_perc'
    cname16 = x1+str(x)+'_discount_quantity_perc'
    cname17 = x1+str(x)+'_otherdisc_quantity_perc'
    cname18 = x1+str(x)+'_coupdisc_price_perc'
    cname19 = x1+str(x)+'_otherdisc_price_perc'
    cname20 = x1+str(x)+'_discount_price_perc'
    
    coup_agg[cname12] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname3]/x[cname1], axis=1)
    coup_agg[cname13] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname5]/x[cname1], axis=1)
    coup_agg[cname14] = coup_agg.apply(lambda x: 0 if (x[cname1]==0) else x[cname7]/x[cname1], axis=1)
    coup_agg[cname15] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname4]/x[cname2], axis=1)
    coup_agg[cname16] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname6]/x[cname2], axis=1)
    coup_agg[cname17] = coup_agg.apply(lambda x: 0 if (x[cname2]==0) else x[cname8]/x[cname2], axis=1)
    coup_agg[cname18] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else x[cname10]/x[cname9], axis=1)
    coup_agg[cname19] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else x[cname11]/x[cname9], axis=1)
    coup_agg[cname20] = coup_agg.apply(lambda x: 0 if (x[cname9]==0) else (x[cname10]+x[cname11])/x[cname9], axis=1)

In [23]:
coup_agg.reset_index().to_csv('coup_agg.csv', index=False)

In [24]:
coup_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1116 entries, 1 to 1116
Columns: 580 entries, coup_trans_count to coup_quarter_4_discount_price_perc
dtypes: float64(575), int64(5)
memory usage: 4.9 MB
